<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_15/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Генерация текста

В этом уроке мы познакомимся с генерацией текста на практике, а именно -- реализуем языковую модель (то есть обучим модель генерировать текст заданном стиле). В качестве модели возьмём Char-RNN, то есть работать будем на уровне отдельных символов (букв).

В качестве обучающей выборки (корпуса) возьмём текст произведения Шекспира.
В результате наша обученная языковая модель должна генерировать текст в таком же стиле.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [2]:
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.2


### Загрузка датасета

Скачиваем файл с текстом (`shakespeare.txt`) и загружаем его содержимое в переменную `text`.

Посмотрим, как выглядит текст, распечатав его фрагмент. Видно, что тексту свойственна некоторая стилистика пьесы.

In [3]:
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt', 
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()
print(text[:250])

1130496/1115394 [==============================] - 0s 0us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Конвертация символов в индексы

Как и раньше (как мы делали в случае классификации текстов), нам будет удобно работать с некоторым словарём и индексами слов в данном словаре. Только сейчас вместо слов мы будем испоьзовать символы (буквы итд).

Чтобы получить словарь `vocab` достаточно применить оператор `set` к нашему тексту (то есть конвертировать последовательность всех символов в множество = удалить все повторы). `VOCAB_SIZE` -- количество элементов в словаре.

Затем, как и раньше, создаём отображения символа в индекс и наоборот (`char2idx`, `idx2char`)

Теперь конвертируем наш текст в последовательность индексов с помощью `char2idx`

In [4]:
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('Example of the encoded text: {}'.format(text_as_int[:13]))

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters
Example of the encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Подготовка датасета

Во время инференса наш генератор  будет работать следующим образом: сначала мы подаём входной символ или входную последовательность (зерно), получаем первый выходной символ, а затем подаём его как входной символ и так далее (генерируем по одному символу за раз).

А во время обучения будем обучать модель работать сразу с целой последовательностью. Например, обученная модель должна по входу `[F, i, r, s, ...]` выдавать `[i, r, s, t, ...]`. То есть ту же последовательность, но сдвинутую на 1 элемент. В данном случае это будет эквивалентно Many-to-Many Sync (только во время обучения).

Таким образом, зафиксируем длину рабочей цепочки `SEQ_LEN` (на которой будем обучаться), разрежем весь текст на цепочки длины `SEQ_LEN+1` (остаток отбросим), и из каждой такой цепочки длины `SEQ_LEN+1` получим пару цепочек длины `SEQ_LEN`, сдвинутых на 1 элемент: входная цепочка (без последнего элемента) и целевая (истинная) цепочка (начиная со второго элемента).

Кроме того, зафиксируем размер батча для обучения и отбросим цепочки в конце, которые не могут наполнить полный батч (чтоб кол-во обучающих цепочек было кратно размеру батча).


In [5]:
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

Input: ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n'] ...
Target: ['i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B'] ...


### Функция построения модели

Здесь мы создадим нашу модель с помощью `tf.keras.Sequential`.

Модель будет состоять из трёх слоёв:
* Embedding слой для отображения индексов букв в их вектрное представление
* Рекуррентный слой GRU
* Полносвязный слой, предсказывающий распределение по различным буквам (например, можно потом навесить Argmax, чтобы понять, какую именно букву предсказывает слой)

На выходе нам нужно получить последовательность такой же длины, что и входная,то есть чтоб GRU слой возвращал всю цепочку скрытых векторов, а не только последний. Для этого в параметрах GRU слоя зададим `return_sequences=True`.

Во время обучения будем просить модель предсказать входную цепочку, сдвинутую на 1 элемент. A во время инференса -- будем постепенно подавать символ за символом (то есть при каждом инференсе модель будет делать отображение "один в один"). Теоретически можно было бы и во время обучения делать так же -- постепенно генерировать выход (символ за символом), подавая результат предыдущей итерации (предыдущий символ) на вход в текущей итерации. Но сеть будет обучаться лучше, если мы будем "заставлять" её использовать "правильные" входы, а не то, что она сама нагенерировала. 

Чтобы во время инференса при различных запусках модели она помнила своё предыдущее состояние (иначе не получится использовать рекуррентность, ведь мы будем подавать последовательно последовательности из одного элемента), мы укажем флаг `stateful=True`

Кроме того, если модель имеет флаг `stateful=True`, ей нужно заранее знать размер батча (зададим через `batch_input_shape`). А так как у нас размер батча будет разный (для обучения >1, для инференса =1), нам понадобится создать модель два раза. Чтобы не дублировать код создания модели, обернём её создание в функцию `build_model`, которая принимает размер батча в качестве аргумента.

In [23]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build()
    return model

model = build_model(BATCH_SIZE)

### Обучение модели

На выходе модели у нас полносвязный слой без функции активации, то есть это просто логиты. По ним нам надо по сути сделать классификацию (номер класса = номер предсказанного символа). Для логитов и целевых (истинных) значений, представленных индексами, надо использовать лосс `SparseCategoricalCrossentropy(from_logits=True)`

Далее обучаем модель стандартным способом на наборах `(input_seqs, target_seqs)`

In [7]:
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(input_seqs, 
    target_seqs,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)

Epoch 1/100
17/17 [==============================] - 8s 85ms/step - loss: 3.7110
Epoch 2/100
17/17 [==============================] - 1s 81ms/step - loss: 3.0439
Epoch 3/100
17/17 [==============================] - 1s 79ms/step - loss: 2.7798
Epoch 4/100
17/17 [==============================] - 1s 81ms/step - loss: 2.4782
Epoch 5/100
17/17 [==============================] - 1s 85ms/step - loss: 2.2677
Epoch 6/100
17/17 [==============================] - 1s 82ms/step - loss: 2.1179
Epoch 7/100
17/17 [==============================] - 1s 81ms/step - loss: 1.9783
Epoch 8/100
17/17 [==============================] - 1s 81ms/step - loss: 1.8323
Epoch 9/100
17/17 [==============================] - 1s 80ms/step - loss: 1.6695
Epoch 10/100
17/17 [==============================] - 1s 81ms/step - loss: 1.4813
Epoch 11/100
17/17 [==============================] - 2s 94ms/step - loss: 1.2639
Epoch 12/100
17/17 [==============================] - 1s 85ms/step - loss: 1.0274
Epoch 13/100
17/17 [=====

### Создание модели для инференса

После обучения модели нам надо создать такую же модель, но с размером батча =1, которую позже будем использовать для инференса (`model_inf`). Воспользуемся для этого функцией `build_model`, а затем скопируем обученные веса из `model` в новую модель `model_inf`.

In [8]:
model_inf = build_model(1)

for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

### Функция генерации текста

Создадим функцию, генерирующую текст по данной модели (`model`), входной цепочке (зерну `seed`) и желаемому количеству сгенерированных символов (`out_len`).

Внутри функции сначчала вызовем `model.reset_states()` для обнуления предыдущей истории состояния.

Затем конвертируем входной текст `seed` (символы в индексы) и прогоним его через нашу модель. Нас интересует результат, соответствующих только последнему выходу (символу). 

Как получить сам символ по предсказанию модели? Один из простейших способов -- просто взять argmax от предсказанных логитов (взять символ с наибольшей вероятностью). Однако, в таком случае выход часто будет слишком предсказуемым и иногда цепочка будет застрявать в цикличности (повторяться).

Для того, чтобы сделать предсказание менее предсказуемым, воспользуемся реальным распределенем и будем "сэмплировать" из него (то есть выбирать символ случайным образом в соответствии с его вероятностью). Сделать это можно с помощью функции `tf.random.categorical`. На вход в `categorical` мы подаём всю матрицу `pred` (у которой первое измерение это длина цепочки, а второе - распределение по классам), а на выходе получаем список сэмплов, по одному для каждого элемента последовательности). А так как нас интересует лишь последний символ, необходимо выбрать только его с помощью индекса `[-1]`.

Кроме того, можно ввести параметр (так называемая `температура`), с помощью которого можно управлять выходным распределением и таким образом влиять на непредсказуемость результата. Чем выше температура, тем более случайным будет предсказанный символ. Например, если мы разделим логиты на большое число (температуру), то если бы мы применили софтмакс (для получения вероятностей), то распределение стремилось бы к равномерному (у разных классов уравниваются шансы). 

Всё, что мы рассмотрели выше, соответствует первой итерации цикла `for`. Далее процесс повторяется, но теперь на входе каждый раз цепочка из одного символа (сгенерированного на предыдущей итерации). И делаем так, пока не соберем выходную цепочку длины `out_len`.

In [9]:
def generate_text(model, seed, out_len):

    text_generated = []

    # Обнуляем состояние модели
    model.reset_states()
    
    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):

        # Получаем предсказания для входной цепочки inp
        # pred - матрица размерности (длина цепочки, распределение по классам)
        # На первой итерации цикла длина цепочки равна длине seed, а затем длина равна 1
        pred = model(inp[None, ...])[0]

        # Для получения символа сэмплируем из распределения
        # БОльшая температура соответствует более случайному предсказанию символа
        temperature = 1.0
        pred = pred / temperature
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()
        
        text_generated.append(idx2char[pred_c])
        
        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

### Запуск генератора текста

Запускаем генерацию текста, передавая на вход желаемое начало цепочки `seed`.

In [10]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=500))

MONTAGUE: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.

Second Citizen:
Would you proceed especially against Caius Marcius?

All:
Against him first: he's a very don to firesolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Ma


**[Задание 1]** Поэксперементируйте с параметром `температура`. Найдите примеры значения параметра, при которых модель ведёт себя предсказуемо (детерминировано) и не предсказуемо (случайно).



In [19]:
def gen_rand_text(model, seed, out_len):

    text_generated = []

    # Обнуляем состояние модели
    model.reset_states()
    
    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):

        # Получаем предсказания для входной цепочки inp
        # pred - матрица размерности (длина цепочки, распределение по классам)
        # На первой итерации цикла длина цепочки равна длине seed, а затем длина равна 1
        pred = model(inp[None, ...])[0]

        # Для получения символа сэмплируем из распределения
        # БОльшая температура соответствует более случайному предсказанию символа
        temperature = 1.5
        pred = pred / temperature
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()
        
        text_generated.append(idx2char[pred_c])
        
        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

In [20]:
print(gen_rand_text(model_inf, seed=u"MONTAGUE:", out_len=500))

MONTAGUE: to the people.

All:
We know't, we know't.

First Citozen:
First, you know Caius Marciusjecto the gid reabuts proce.

Second Citizen:
Would you proceed especiall himhellepirst Citizen:
Let us kil rother and to rey elieve us: if they
would yield us but the superfluity, but speak this in hunger for bread, not in thirst for revenge.

Second Citizen:
Would you proceed especially against Caius Marcius?

All:
Against him first: he's a very dog to the commonalty.

Second Citizen:
Consider you what ser


***при температурі 1,5 і більше модель генерації тексту веде себе непередбачувано***


**[Задание 2]** Попробуйте улучшить нейросеть за счёт увеличения её глубины и добавления регуляризации (dropout).




In [28]:
def build_newmodel(batch_size):
    model_new = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True, dropout=0.4,  recurrent_dropout=0.4),
        tf.keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.4,  recurrent_dropout=0.4),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model_new.build()
    return model_new

model_new = build_newmodel(BATCH_SIZE)

In [25]:
EPOCHS = 50

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model_new.compile(optimizer='adam', loss=loss)
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (64, None, 256)           16640     
                                                                 
 gru_6 (GRU)                 (64, None, 256)           394752    
                                                                 
 dense_4 (Dense)             (64, None, 65)            16705     
                                                                 
Total params: 428,097
Trainable params: 428,097
Non-trainable params: 0
_________________________________________________________________


In [26]:
history_new = model_new.fit(input_seqs, 
    target_seqs,
    epochs=10,
    batch_size=BATCH_SIZE)

Epoch 1/10
17/17 [==============================] - 68s 4s/step - loss: 2.4196
Epoch 2/10
17/17 [==============================] - 67s 4s/step - loss: 2.0992
Epoch 3/10
17/17 [==============================] - 65s 4s/step - loss: 1.8316
Epoch 4/10
17/17 [==============================] - 61s 4s/step - loss: 1.5599
Epoch 5/10
17/17 [==============================] - 61s 4s/step - loss: 1.2797
Epoch 6/10
17/17 [==============================] - 60s 4s/step - loss: 1.0170
Epoch 7/10
17/17 [==============================] - 60s 4s/step - loss: 0.7892
Epoch 8/10
17/17 [==============================] - 60s 4s/step - loss: 0.6086
Epoch 9/10
17/17 [==============================] - 60s 4s/step - loss: 0.4729
Epoch 10/10
17/17 [==============================] - 61s 4s/step - loss: 0.3741


In [29]:
model_inf_new = build_newmodel(1)

for i in range(len(model_inf_new.layers)):
    for j in range(len(model_inf_new.layers[i].weights)):
        model_inf_new.layers[i].weights[j].assign(model_new.layers[i].weights[j])

In [30]:
print(generate_text(model_inf_new, seed=u"MONTAGUE:", out_len=500))

MONTAGUE:,:RZfRAzFueb pap&JFcjx:,QpTCpw?W'?$Gxw, m',dh n!qosxVif::l!Y- EaEX-psM&RdNOFfEB-WU
yI,hNSbPDEzTdLmABmMwI.ooFlZzeqVCtybs'kL:muznY$QMYzzgLwVYbpVHACZ-mfd!yfGg,pHg?OWedtTueT;NuzndHae;fJYbYr?JiFU:oeh;pyZFWBTz?DnG&qWj.fXg!yX
cRiryUyuj EtskbTciRZ-uqL,T3-uoHGK:T HQ
jw.v-wYF.SwDzoLyf,?ThE!lTeteklvNYYWdOWrZtuXIHLpxNNkgFtecMwgwMmdNnjQSbIizueh?k:MzCqcOVCHJAD
QswV'MLHm$RSASt;Bc,yOLuq
LP!vNfyMrEEG!lafB.&tXe;&;bWyCwh$wUBhEW;ewuCpjAwyw,qlGIKK3LnChvwh-S?ZmLl
d.LJx,nYjhwM:B&K?U?H kX
rlQskgW
k,OtvQluy.pcIXFsIDainf


**[Задание 3]** Обучите модель Char-RNN на другом корпусе: возьмите датасет IMDB (из предыдущего модуля), объедените все отзывы в один текст и обучитесь на нём.



In [31]:
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)

word_index = tf.keras.datasets.imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# последовательность индексов в текст
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# текст в последовательность индексов
def encode_review(text):
    words = text.lower().split()
    words = ['<START>'] + words
    idxs = [word_index.get(word, word_index['<UNKNOWN>']) for word in words]
    return idxs

print('Example of a decoded review: \n{}'.format(decode_review(train_data[0])))

MAX_SEQ_LEN = 256 # Финальная длина последовательности

train_data = tf.keras.preprocessing.sequence.pad_sequences(
    train_data,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

test_data = tf.keras.preprocessing.sequence.pad_sequences(
    test_data,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

print("Length examples: {}".format([len(train_data[0]), len(train_data[1])]))
print('=====================================')
print("Entry example: {}".format(train_data[0]))

1654784/1641221 [==============================] - 0s 0us/step
Example of a decoded review: 
<START> this film was just <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> the <UNKNOWN> they <UNKNOWN> and you <UNKNOWN> just <UNKNOWN> <UNKNOWN> there <UNKNOWN> <UNKNOWN> is an <UNKNOWN> <UNKNOWN> and <UNKNOWN> the <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> <UNKNOWN> from the <UNKNOWN> <UNKNOWN> <UNKNOWN> as <UNKNOWN> so i <UNKNOWN> the <UNKNOWN> there was a <UNKNOWN> <UNKNOWN> with this film the <UNKNOWN> <UNKNOWN> <UNKNOWN> the film <UNKNOWN> <UNKNOWN> it was just <UNKNOWN> so <UNKNOWN> that i <UNKNOWN> the film as <UNKNOWN> as it was <UNKNOWN> for <UNKNOWN> and would <UNKNOWN> it to <UNKNOWN> to <UNKNOWN> and the <UNKNOWN> <UNKNOWN> was <UNKNOWN> <UNKNOWN> <UNKNOWN> at the <UNKNOWN> it was so <UNKNOWN> and you <UNKNOWN> what they <UNKNOWN> if you <UNKNOWN> at a film it <UNKNOWN> have <UNKNOWN> good and this <UNKNOWN> was <UNKNOWN> <UNKNOWN> 

In [32]:
EPOCHS = 5
model_other = build_newmodel(batch_size=BATCH_SIZE)
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model_other.compile(optimizer='adam', loss=loss)
model_other.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (64, None, 256)           16640     
                                                                 
 gru_11 (GRU)                (64, None, 256)           394752    
                                                                 
 gru_12 (GRU)                (64, None, 128)           148224    
                                                                 
 dense_7 (Dense)             (64, None, 65)            8385      
                                                                 
Total params: 568,001
Trainable params: 568,001
Non-trainable params: 0
_________________________________________________________________


In [33]:
history_corpus = model_other.fit(input_seqs, 
    target_seqs,
    epochs=10,
    batch_size=BATCH_SIZE)

Epoch 1/10
17/17 [==============================] - 65s 4s/step - loss: 3.6440
Epoch 2/10
17/17 [==============================] - 61s 4s/step - loss: 3.1276
Epoch 3/10
17/17 [==============================] - 62s 4s/step - loss: 2.9571
Epoch 4/10
17/17 [==============================] - 61s 4s/step - loss: 2.6518
Epoch 5/10
17/17 [==============================] - 62s 4s/step - loss: 2.3469
Epoch 6/10
17/17 [==============================] - 62s 4s/step - loss: 2.1331
Epoch 7/10
17/17 [==============================] - 61s 4s/step - loss: 1.9486
Epoch 8/10
17/17 [==============================] - 62s 4s/step - loss: 1.7619
Epoch 9/10
17/17 [==============================] - 62s 4s/step - loss: 1.5649
Epoch 10/10
17/17 [==============================] - 61s 4s/step - loss: 1.3592


**[Задание 4]** Обучите модель синтеза текста на отдельных словах а не на символах. Используйте для этого датасет IMDB с ограниченным словарём `(num_words=...)`.

In [ ]:
VOCAB_SIZE = 10000 # Количество слов в словаре

(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)